In [1]:
!pip install uszipcode

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from uszipcode import SearchEngine
# import us

/usr/local/lib/python3.7/dist-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [3]:
from google.colab import drive
drive.mount("/drive")
# df = pd.read_csv("/drive/My Drive/dataopen/LC/Lending_Club_Accepted_2014_2018.csv", delimiter = ',') 

Drive already mounted at /drive; to attempt to forcibly remount, call drive.mount("/drive", force_remount=True).


In [4]:
accepted = pd.read_csv("/drive/MyDrive/Summer_2022_Datathon/LC/Lending_Club_Accepted_2014_2018.csv", delimiter = ',') 
# rejected = pd.read_csv("/drive/MyDrive/Summer_2022_Datathon/LC/Lending_Club_Rejected_2014_2018.csv", delimiter = ',', usecols=['Application_Date', 'Zip_Code'])
# immigration_dataset = pd.read_csv('/drive/My Drive/dataopen/met-immigration.csv', usecols=['NAME', 'INTERNATIONALMIG2021','NETMIG2021','INTERNATIONALMIG2020'], encoding='unicode_escape') 
immigration_dataset = pd.read_csv( '/drive/MyDrive/Summer_2022_Datathon/Continent_data.csv',usecols=['Imm% tot pop', 'Metro N Imm', 'Round total MSA population'], delimiter = '\t', encoding='UTF-16') 
name = pd.read_csv( '/drive/MyDrive/Summer_2022_Datathon/Continent_data.csv', usecols=['Metro'],delimiter = '\t', encoding='UTF-16')                  
uszips = pd.read_csv('/drive/MyDrive/Summer_2022_Datathon/archive/uszips.csv')
# uszips = pd.read_csv('/drive/MyDrive/dataopen/uszips.csv')

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: DtypeWarning: Columns (19,59,118,129,130,131,134,135,136,139,145,146,147) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [5]:
accepted['year'] = [int(x.split('-')[1]) for x in accepted['issue_d']]
accepted['zip_code'] = [str(x)[:3] for x in accepted['zip_code']]

In [6]:
accepted['annual_inc']

0           55000.0
1           65000.0
2           63000.0
3          110000.0
4          104433.0
             ...   
2029947     95000.0
2029948    108000.0
2029949    227000.0
2029950    110000.0
2029951     95000.0
Name: annual_inc, Length: 2029952, dtype: float64

In [7]:
immigration_dataset=pd.concat([immigration_dataset,name], axis=1)
immigration_dataset=immigration_dataset.sort_values(by=['Imm% tot pop'], ascending=False)

In [8]:
accepted_pre = accepted[accepted['year'] < 2016]
accepted_post = accepted[accepted['year'] >= 2016]
accepted_pre.set_index(['issue_d'], append=True)
accepted_pre.set_index('zip_code', append=True)
accepted_post.set_index('issue_d')

,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,...,hardship_last_payment_amount,disbursement_method,debt_settlement_flag,debt_settlement_flag_date,settlement_status,settlement_date,settlement_amount,settlement_percentage,settlement_term,year
issue_d,,,,,,,,,,,,,,,,,,,,,
Mar-2018,130954621,NaN,5000.0,5000.0,5000.0,36 months,20.39,186.82,D,D4,...,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN,2018
Mar-2018,130964697,NaN,15000.0,15000.0,15000.0,36 months,9.92,483.45,B,B2,...,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN,2018
Mar-2018,130955326,NaN,11200.0,11200.0,11200.0,60 months,30.79,367.82,G,G1,...,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN,2018
Mar-2018,130504052,NaN,25000.0,25000.0,25000.0,60 months,21.85,688.35,D,D5,...,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN,2018
Mar-2018,130956066,NaN,3000.0,3000.0,3000.0,36 months,7.34,93.10,A,A4,...,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN,2018
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Oct-2016,89885898,NaN,24000.0,24000.0,24000.0,60 months,12.79,543.50,C,C1,...,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN,2016
Oct-2016,88977788,NaN,24000.0,24000.0,24000.0,60 months,10.49,515.74,B,B3,...,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN,2016
Oct-2016,88985880,NaN,40000.0,40000.0,40000.0,60 months,10.49,859.56,B,B3,...,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN,2016


In [9]:
generated_zips = []
zip_base = set()
for place in immigration_dataset['Metro']:
    city_names = place.split(', ')[0].split('-')
    state_name = place.split(', ')[1]
    in_state = uszips[uszips.state_id == state_name]
    city_zips = set()
    # city_zips = ''
    for city_name in city_names:
      city_zips = city_zips.union(set([str(x)[:3] for x in in_state[in_state.city == city_name].zip]))
    county_name_zips = set([str(x)[:3] for x in in_state[in_state.county_name == city_name].zip])
    zips = city_zips.union(county_name_zips)
    generated_zips.append(zips)
    zip_base = zip_base.union(zips)

immigration_dataset['zips'] = generated_zips

In [10]:
imm_pre = immigration_dataset
imm_post = immigration_dataset

In [11]:
imm_pre.head()

,Imm% tot pop,Metro N Imm,Round total MSA population,Metro,zips
250,0.406993,2478853,6091000,"Miami-Fort Lauderdale-Pompano Beach, FL","{331, 333, 330}"
114,0.385842,766995,1988000,"San Jose-Sunnyvale-Santa Clara, CA","{940, 951, 950, 943}"
208,0.331390,15403,46000,"Nogales, AZ",{856}
271,0.330924,4384613,13250000,"Los Angeles-Long Beach-Anaheim, CA","{900, 928, 908}"
433,0.314917,18320,58000,"Eagle Pass, TX",{788}


In [12]:
imm_post.head()

,Imm% tot pop,Metro N Imm,Round total MSA population,Metro,zips
250,0.406993,2478853,6091000,"Miami-Fort Lauderdale-Pompano Beach, FL","{331, 333, 330}"
114,0.385842,766995,1988000,"San Jose-Sunnyvale-Santa Clara, CA","{940, 951, 950, 943}"
208,0.331390,15403,46000,"Nogales, AZ",{856}
271,0.330924,4384613,13250000,"Los Angeles-Long Beach-Anaheim, CA","{900, 928, 908}"
433,0.314917,18320,58000,"Eagle Pass, TX",{788}


In [13]:
existing_zip_pre = set(accepted_pre.zip_code)
existing_zip_post = set(accepted_post.zip_code)

In [14]:
l = len(immigration_dataset)
decilep = imm_pre.iloc[int(0.1*l)]['Imm% tot pop']
decilepost = imm_post.iloc[int(0.1*l)]['Imm% tot pop']

In [15]:
imm_pre['Immigrant90'] = immigration_dataset['Imm% tot pop'].apply(lambda x: 1 if x >= decilep else 0)
imm_pre['Immigrant90'] = immigration_dataset['Imm% tot pop'].apply(lambda x: 1 if x >= decilepost else 0)

In [16]:
imm_pre = imm_pre[imm_pre['zips'].apply(lambda x: len(x.intersection(existing_zip_pre)) != 0)]
imm_post = imm_post[imm_post['zips'].apply(lambda x: len(x.intersection(existing_zip_post)) != 0)]
imm_pre.zips = imm_pre['zips'].apply(lambda x: x.intersection(existing_zip_pre))
imm_post.zips = imm_post['zips'].apply(lambda x: x.intersection(existing_zip_post))

/usr/local/lib/python3.7/dist-packages/pandas/core/generic.py:5516: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


In [17]:
len(imm_pre), len(imm_post)

(514, 510)

In [18]:
zips_meanp = accepted_pre.groupby(['zip_code']).int_rate.mean()
zips_meanpo = accepted_post.groupby(['zip_code']).int_rate.mean()

In [19]:
ficopl = accepted_pre.groupby(['zip_code']).fico_range_low.mean()
ficopol = accepted_post.groupby(['zip_code']).fico_range_low.mean()
ficoph = accepted_pre.groupby(['zip_code']).fico_range_high.mean()
ficopoh = accepted_post.groupby(['zip_code']).fico_range_high.mean()

In [24]:
income_meanp = accepted_pre.groupby(['annual_inc']).int_rate.mean()
income_meanpo = accepted_post.groupby(['annual_inc']).int_rate.mean()

In [20]:
imm_pre.head()

,Imm% tot pop,Metro N Imm,Round total MSA population,Metro,zips,Immigrant90
250,0.406993,2478853,6091000,"Miami-Fort Lauderdale-Pompano Beach, FL","{331, 333, 330}",1
114,0.385842,766995,1988000,"San Jose-Sunnyvale-Santa Clara, CA","{940, 951, 950, 943}",1
208,0.331390,15403,46000,"Nogales, AZ",{856},1
271,0.330924,4384613,13250000,"Los Angeles-Long Beach-Anaheim, CA","{900, 928, 908}",1
433,0.314917,18320,58000,"Eagle Pass, TX",{788},1


In [22]:
import statistics
imm_pre['Int'] = imm_pre['zips'].apply(lambda x: statistics.mean([zips_meanp[i] for i in x]))
imm_post['Int'] = imm_post['zips'].apply(lambda x: statistics.mean([zips_meanpo[i] for i in x]))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [23]:
imm_pre['fico'] = imm_pre['zips'].apply(lambda x: statistics.mean([(ficoph[i]+ficopl[i])/2 for i in x]))
imm_post['fico'] = imm_post['zips'].apply(lambda x: statistics.mean([(ficopoh[i]+ficopol[i])/2 for i in x]))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [ ]:
# pct_imm = []
# for place in accepted['zip_code']:
#   if place in zip_base: 
#     for i, zips in enumerate(immigration_dataset['zips']):
#       if place in zips:
#         pct_imm.append(immigration_dataset.iloc[i]['Imm% tot pop'])
#         break
#   else:
#     pct_imm.append(None)

In [ ]:
accepted_pre.head().issue_d

0    Dec-2015
1    Dec-2015
2    Dec-2015
3    Dec-2015
4    Dec-2015
Name: issue_d, dtype: object

### Research Question: How does immigration rate of a city affect interest rate? 

In [ ]:
dates = accepted_pre.index.get_level_values(0).to_list()
accepted_pre['issue_d'] = pd.Categorical(dates)


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


PooledOLS and check required assumptions

In [ ]:
pip install linearmodels 

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
pip install statsmodels

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
accepted_pre['issue_d']

0                0
1                1
2                2
3                3
4                4
            ...   
1352670    1352670
1352671    1352671
1352672    1352672
1352673    1352673
1352674    1352674
Name: issue_d, Length: 656724, dtype: category
Categories (656724, int64): [0, 1, 2, 3, ..., 1352671, 1352672, 1352673, 1352674]

In [ ]:
# Perform PooledOLS
# from linearmodels import PooledOLS
# import statsmodels.api as sm
# exog = sm.tools.tools.add_constant(np.array(zip(accepted_pre['issue_d'].to_list(), accepted_pre['pct_imm'].to_list())))
# endog = np.array(accepted_pre['int_rate'].to_list())
# mod = PooledOLS(endog, exog)
# pooledOLS_res = mod.fit(cov_type='clustered', cluster_entity=True)
# # Store values for checking homoskedasticity graphically
# fittedvals_pooled_OLS = pooledOLS_res.predict().fitted_values
# residuals_pooled_OLS = pooledOLS_res.resids

In [ ]:
accepted_pre.set_index(['issue_d'])
accepted_pre.set_index(['zip_code'])

,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,...,hardship_last_payment_amount,disbursement_method,debt_settlement_flag,debt_settlement_flag_date,settlement_status,settlement_date,settlement_amount,settlement_percentage,settlement_term,year
zip_code,,,,,,,,,,,,,,,,,,,,,
190,68407277,NaN,3600.0,3600.0,3600.0,36 months,13.99,123.03,C,C4,...,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN,2015
577,68355089,NaN,24700.0,24700.0,24700.0,36 months,11.99,820.28,C,C1,...,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN,2015
605,68341763,NaN,20000.0,20000.0,20000.0,60 months,10.78,432.66,B,B4,...,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN,2015
076,66310712,NaN,35000.0,35000.0,35000.0,60 months,14.85,829.90,C,C5,...,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN,2015
174,68476807,NaN,10400.0,10400.0,10400.0,60 months,22.45,289.91,F,F1,...,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN,2015
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
773,8598660,NaN,18400.0,18400.0,18400.0,60 months,14.47,432.64,C,C2,...,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN,2014
377,9684700,NaN,22000.0,22000.0,22000.0,60 months,19.97,582.50,D,D5,...,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN,2014
913,9604874,NaN,2000.0,2000.0,2000.0,36 months,7.90,62.59,A,A4,...,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN,2014


In [ ]:
imm_pre.columns

Index(['Imm% tot pop', 'Metro N Imm', 'Round total MSA population', 'Metro',
       'zips', 'Immigrant90', 'Int', 'fico'],
      dtype='object')

In [ ]:
imm_pre['zipstr'] = imm_pre.zips.apply(lambda x: list(x)[0])

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [ ]:
imm_pre['ficostr'] = imm_pre['fico'].apply(lambda x: str(x))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [ ]:
imm_pre.head()

,Imm% tot pop,Metro N Imm,Round total MSA population,Metro,zips,Immigrant90,Int,fico,zipstr,ficostr
250,0.406993,2478853,6091000,"Miami-Fort Lauderdale-Pompano Beach, FL","{331, 330, 333}",1,13.069119,693.755485,331,693.7554853928906
114,0.385842,766995,1988000,"San Jose-Sunnyvale-Santa Clara, CA","{950, 951, 940, 943}",1,12.646128,696.811703,950,696.8117033860568
208,0.331390,15403,46000,"Nogales, AZ",{856},1,12.960563,696.554974,856,696.5549738219895
271,0.330924,4384613,13250000,"Los Angeles-Long Beach-Anaheim, CA","{908, 900, 928}",1,12.921089,694.694527,908,694.6945271654282
433,0.314917,18320,58000,"Eagle Pass, TX",{788},1,13.127181,696.669604,788,696.669603524229


In [ ]:
import statsmodels.api as sm

# FE und RE model
from linearmodels import PanelOLS
from linearmodels import RandomEffects
imm_pre = imm_pre.set_index(['zipstr', 'fico'])
exog = sm.tools.tools.add_constant(imm_pre['Imm% tot pop'])
endog = imm_pre['Int']
# random effects model
model_re = RandomEffects(endog, exog) 
re_res = model_re.fit() 
# fixed effects model
model_fe = PanelOLS(endog, exog, entity_effects = True) 
fe_res = model_fe.fit() 
#print results
print(re_res)
print(fe_res)

                        RandomEffects Estimation Summary                        
Dep. Variable:                    Int   R-squared:                        0.9370
Estimator:              RandomEffects   R-squared (Between):             -0.0016
No. Observations:                 514   R-squared (Within):               0.0078
Date:                Sun, Jul 24 2022   R-squared (Overall):             -0.0015
Time:                        01:04:44   Log-likelihood                    547.92
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      7609.2
Entities:                         378   P-value                           0.0000
Avg Obs:                       1.3598   Distribution:                   F(1,512)
Min Obs:                       1.0000                                           
Max Obs:                       5.0000   F-statistic (robust):             0.4433
                            